<a href="https://colab.research.google.com/github/selveszero/530_final_project/blob/master/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!mkdir data
!wget http://msvocds.blob.core.windows.net/annotations-1-0-3/captions_train-val2014.zip -P ./data/
!wget http://images.cocodataset.org/zips/train2014.zip -P ./data/
!wget http://images.cocodataset.org/zips/val2014.zip -P ./data/

!unzip -q ./data/captions_train-val2014.zip -d ./data/
!rm ./data/captions_train-val2014.zip
!unzip -q ./data/train2014.zip -d ./data/
!rm ./data/train2014.zip 
!unzip -q ./data/val2014.zip -d ./data/ 
!rm ./data/val2014.zip 

In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
!python build_vocab.py   


In [0]:
!python resize.py 

The following cell trains 1 epoch of the training data.
More tunable parameters can be seen in train.py

In [70]:
!python train.py --num_epochs=1 --model_path='vgg_models'

Namespace(batch_size=128, caption_path='data/annotations/captions_train2014.json', crop_size=224, embed_size=256, hidden_size=512, image_dir='data/resized2014', learning_rate=0.001, log_step=100, model_path='vgg_models', num_epochs=1, num_layers=1, num_workers=2, save_step=1000, vocab_path='data/vocab.pkl')
loading annotations into memory...
Done (t=0.99s)
creating index...
index created!
Epoch [0/1], Step [0/3236], Loss: 9.2146, Perplexity: 10042.8652
Epoch [0/1], Step [100/3236], Loss: 3.8116, Perplexity: 45.2245
Epoch [0/1], Step [200/3236], Loss: 3.4982, Perplexity: 33.0559
Epoch [0/1], Step [300/3236], Loss: 3.2787, Perplexity: 26.5406
Epoch [0/1], Step [400/3236], Loss: 3.0367, Perplexity: 20.8370
Epoch [0/1], Step [500/3236], Loss: 2.8681, Perplexity: 17.6043
Epoch [0/1], Step [600/3236], Loss: 2.8886, Perplexity: 17.9688
Epoch [0/1], Step [700/3236], Loss: 2.9195, Perplexity: 18.5321
Epoch [0/1], Step [800/3236], Loss: 2.9065, Perplexity: 18.2925
Epoch [0/1], Step [900/3236], L

Uncomment the next cell and change image_path to generate caption for one image.

In [75]:

!python sample.py --image='red_house.png' --encoder_path='vgg_models/encoder-1-1000.ckpt' --decoder_path='vgg_models/decoder-1-1000.ckpt' 
!python sample.py --image='red_house.png' --encoder_path='vgg_models/encoder-1-2000.ckpt' --decoder_path='vgg_models/decoder-1-2000.ckpt' 
!python sample.py --image='red_house.png' --encoder_path='vgg_models/encoder-1-3000.ckpt' --decoder_path='vgg_models/decoder-1-3000.ckpt' 

<start> a man in a red shirt is sitting on a bench . <end>
<start> a man is standing on a bench in front of a building . <end>
<start> a man is sitting on a bench with a dog . <end>


The following cells will compute bleu scores on the validation set.



In [0]:
# resize all validation images and store into resizedval2014

!python resize.py --image_dir='data/val2014' --output_dir='data/resizedval2014'

In [0]:
# change encoder_path and decoder_path to test the effect of different states of the models
encoder_path = 'vgg_models/encoder-1-1000.ckpt'
decoder_path = 'vgg_models/decoder-1-1000.ckpt'

vocab_path = 'data/vocab.pkl'

imagedir = 'data/resizedval2014/'
caption_path = 'data/annotations/captions_val2014.json'

embed_size = 256
hidden_size = 512
num_layers = 1

import torch
import matplotlib.pyplot as plt
import numpy as np 
import pickle 
import os
from torchvision import transforms 
from build_vocab import Vocabulary
from model import EncoderCNN, DecoderRNN
from PIL import Image

from pycocotools.coco import COCO

from nltk.translate.bleu_score import corpus_bleu

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def load_image(image_path, transform=None):
    image = Image.open(image_path)
#     image = image.resize([224, 224], Image.LANCZOS)
    
    image = transform(image).unsqueeze(0)
    return image
  
def Compute_bleu():
  sum_bleu = 0
  coco = COCO(caption_path)
  ann_ids = list(coco.anns.keys())
  print(len(ann_ids))
  counter = 0
  for ann_id in ann_ids:
    # Image preprocessing
    transform = transforms.Compose([
        transforms.ToTensor(), 
        transforms.Normalize((0.485, 0.456, 0.406), 
                             (0.229, 0.224, 0.225))])
    
    # Load vocabulary wrapper
    with open(vocab_path, 'rb') as f:
        vocab = pickle.load(f)

    # Build models
    encoder = EncoderCNN(embed_size).eval()  # eval mode (batchnorm uses moving mean/variance)
    decoder = DecoderRNN(embed_size, hidden_size, len(vocab), num_layers)
    encoder = encoder.to(device)
    decoder = decoder.to(device)

    # Load the trained model parameters
    encoder.load_state_dict(torch.load(encoder_path))
    decoder.load_state_dict(torch.load(decoder_path))

    # Prepare an image
    img_id = coco.anns[ann_id]['image_id']
    path = coco.loadImgs(img_id)[0]['file_name']
    image_path = imagedir + path
    
    image = load_image(image_path, transform)
    image_tensor = image.to(device)
    
    # Generate an caption from the image
    feature = encoder(image_tensor)
    sampled_ids = decoder.sample(feature)
    sampled_ids = sampled_ids[0].cpu().numpy()          # (1, max_seq_length) -> (max_seq_length)
    
    # Convert word_ids to words
    sampled_caption = []
    for word_id in sampled_ids:
        word = vocab.idx2word[word_id]
        sampled_caption.append(word)
        if word == '<end>':
            break
    sentence = ' '.join(sampled_caption)
    
    
    # get reference captions
    cap = coco.anns[ann_id]['caption']
    tokens = nltk.tokenize.word_tokenize(str(cap).lower())
    caption = []
    caption.append('<start>')
    caption.extend([token for token in tokens])
    caption.append('<end>')
    
    references = [caption]
    hypotheses = [sentence]
#     print(references)
    bleu = corpus_bleu(references, hypotheses, weights=(1.0, 0, 0, 0))
    sum_bleu += bleu
#     print(bleu)
    counter += 1
  
    if counter % 10 == 0:
      print('val size:', counter)
      print('net bleu:', sum_bleu/counter)
      print('Ex reference:', ' '.join(caption))
      print('Ex hypothesis:', sentence)
      print('Ex image:', image_path)
      
  print(sum_bleu/len(ann_ids))
    
    # Print out the image and the generated caption
#     print (sentence)
#     image = Image.open(image)
#     plt.imshow(np.asarray(image))



In [14]:
Compute_bleu()

loading annotations into memory...
Done (t=0.30s)
creating index...
index created!
202654
val size: 10
net bleu: 0.3531404720435889
Ex reference: <start> the bike has a clock as a tire . <end>
Ex hypothesis: <start> a person sitting on a bench in a park . <end>
Ex image: data/resizedval2014/COCO_val2014_000000203564.jpg
val size: 20
net bleu: 0.3363080518469441
Ex reference: <start> two women waiting at a bench next to a street . <end>
Ex hypothesis: <start> a man in a red shirt is riding a motorcycle . <end>
Ex image: data/resizedval2014/COCO_val2014_000000517069.jpg
val size: 30
net bleu: 0.33739377647797725
Ex reference: <start> city street with parked cars and a bench . <end>
Ex hypothesis: <start> a motorcycle parked next to a car parked next to a building . <end>
Ex image: data/resizedval2014/COCO_val2014_000000016977.jpg
val size: 40
net bleu: 0.3369011649695255
Ex reference: <start> a beautiful dessert waiting to be shared by two people <end>
Ex hypothesis: <start> a plate with

KeyboardInterrupt: ignored